In [1]:
import openai
import os

In [2]:
from openai import OpenAI

In [6]:
client = OpenAI()

In [7]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": "is it too late to join the course?"}]
)

In [8]:
response.choices[0].message.content

"Whether it is too late to join a course depends on several factors, including the specific course's enrollment deadlines, the institution's policies, and the current point in the academic term or program schedule. Here are a few steps you can take to find out:\n\n1. **Check the Course Schedule:** Look at the academic calendar or course schedule on the institution's website to see the enrollment deadlines.\n\n2. **Contact the Instructor:** Reach out to the course instructor to ask if late enrollment is an option. Sometimes, instructors allow late joiners if they can catch up.\n\n3. **Contact the Registrar or Admissions Office:** They will have the most accurate and up-to-date information regarding enrollment policies and deadlines.\n\n4. **Look for Late-Start Classes:** Some institutions offer late-start classes or rolling admissions, which could provide an alternative way to join the course.\n\n5. **Explore Online Options:** If it’s too late to join an in-person class, many institutio

In [1]:
import requests 
import tiktoken

from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

In [12]:
# Fetch documents
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [15]:
es_client = Elasticsearch('http://localhost:9200') 

In [17]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [ ]:
index_name = "course-questions-homework1"
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [ ]:
query = 'How do I execute a command in a running docker container?'

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append({
            "score": hit['_score'],
            "source": hit['_source']})
    
    return result_docs

In [ ]:
result = elastic_search(query)
result

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
result = elastic_search(query)
result

In [ ]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_entries = []
    
    for doc in search_results:
        context_entry = context_template.format(question=doc['question'], text=doc['text'])
        context_entries.append(context_entry)
    
    context = "\n\n".join(context_entries)
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
result

In [ ]:
# Example usage
query = "How do I execute a command in a running docker container?"
prompt = build_prompt(query, result)
print(prompt)

In [ ]:
len(prompt)

In [ ]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
tokens = encoding.encode(prompt)
num_tokens = len(tokens)
num_tokens